Ensemble hard-voting

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 71.5 MB/s 


In [4]:
import pandas as pd

a = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KoBigBird-klue_result2.csv')
a = a.drop(['Context', 'Question'], axis=1)
a = a.drop(0, axis=0)
a = a.rename(columns={'Predicted':'Predicted_a', 'levenshtein':'levenshtein_a'})
a

,Id,Predicted_a,Start,End,Score
1,906631384e91493ebe1c7f34aea6f241,독일 뒤셀도르프로,556.0,565.0,0.725427
2,35e61dcb479643448a2cb7d326ae50a6,페이스북이,543.0,548.0,0.556682
3,075e761b370040cb9041eecd39afc27c,페이스북이,543.0,548.0,0.500647
4,e67ed38f3dd944be94d5b4c53731f334,마드리드에,408.0,413.0,0.998218
5,80cacfdfe76442b0b39568e8b193d066,국제 원자재값 하락은,824.0,835.0,0.588479
...,...,...,...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,2만달러로,546.0,551.0,0.242327
4004,cc7f826b66724ce9b39e3a974ca15661,중동 건설 현장으로,470.0,480.0,0.512491
4005,3282034aa41e4fab980851ffd4a868dd,아시아~유럽 노선의,383.0,393.0,0.589738
4006,0a73550b36df4baf82ac2f98619d22e7,사흘간,313.0,316.0,0.602099


In [5]:
from konlpy.tag import Okt
dfa = a.copy()

long_cnt = 0
low_score = 0
remove_josa = 0
for index, row in dfa.iterrows():
  #예측이 20문자가 넘는다면 앞의 10문자만 사용
  '''
  if len(row["Predicted_a"]) > 20:
    dfa.at[index,'Predicted_a'] = row["Predicted_a"][:10]
    long_cnt +=1
    continue
  '''
  #score가 0.05점이 넘지 않는다면 빈칸으로 대체

  #예측의 마지막이 조사라면 조사를 제거
  tag = Okt().pos(row["Predicted_a"])
  if tag[-1][1] == 'Josa':
    target = tag[-1][0]
    clean = row["Predicted_a"].rstrip(target)
    dfa.at[index,'Predicted_a'] = clean
    remove_josa +=1

print("길어서 제거된 답 ",long_cnt)
print("score가 낮아서 제거된 답", low_score)
print("제거된 조사 수", remove_josa)

길어서 제거된 답  0
score가 낮아서 제거된 답 0
제거된 조사 수 1697


In [6]:
dfa

,Id,Predicted_a,Start,End,Score
1,906631384e91493ebe1c7f34aea6f241,독일 뒤셀도르프,556.0,565.0,0.725427
2,35e61dcb479643448a2cb7d326ae50a6,페이스북,543.0,548.0,0.556682
3,075e761b370040cb9041eecd39afc27c,페이스북,543.0,548.0,0.500647
4,e67ed38f3dd944be94d5b4c53731f334,마드리드,408.0,413.0,0.998218
5,80cacfdfe76442b0b39568e8b193d066,국제 원자재값 하락,824.0,835.0,0.588479
...,...,...,...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,2만달러로,546.0,551.0,0.242327
4004,cc7f826b66724ce9b39e3a974ca15661,중동 건설 현장,470.0,480.0,0.512491
4005,3282034aa41e4fab980851ffd4a868dd,아시아~유럽 노선,383.0,393.0,0.589738
4006,0a73550b36df4baf82ac2f98619d22e7,사흘간,313.0,316.0,0.602099


In [7]:
b = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/koelectra-base-v3-discriminator-finetuned-klue_check(3.46).csv')
b = b.drop(['Context', 'Question', 'guid','levenshtein'], axis=1)
b = b.rename(columns={'Predicted':'Predicted_b'})
b

,Predicted_b,Start,End,Score,Answer,Id
0,뉴 740Li 25주년 에디션,666.0,682.0,0.103343,"['뉴 740Li 25주년 에디션', '뉴 740Li 25주년']",d14cb73158624cf094c546d856fd3c80
1,뒤셀도르프,559.0,565.0,0.572797,['독일 뒤셀도르프'],906631384e91493ebe1c7f34aea6f241
2,꾸준히,1071.0,1074.0,0.981770,['페이스북'],35e61dcb479643448a2cb7d326ae50a6
3,꾸준히,1071.0,1074.0,0.983212,['페이스북'],075e761b370040cb9041eecd39afc27c
4,마드리드,408.0,413.0,0.980107,['마드리드'],e67ed38f3dd944be94d5b4c53731f334
...,...,...,...,...,...,...
4004,2만달러로,546.0,551.0,0.118583,['200만엔'],05fcb8054dc44dab8683579c2cf5e465
4005,중동 건설 현장,470.0,480.0,0.677122,"['중동 건설 현장', '중동']",cc7f826b66724ce9b39e3a974ca15661
4006,아시아~유럽,383.0,389.0,0.535192,"['아시아~유럽 노선', '아시아~유럽']",3282034aa41e4fab980851ffd4a868dd
4007,12일까지,988.0,993.0,0.612616,['사흘'],0a73550b36df4baf82ac2f98619d22e7


In [8]:
dfb = b.copy()

long_cnt = 0
low_score = 0
remove_josa = 0
for index, row in dfb.iterrows():
  #예측이 20문자가 넘는다면 앞의 10문자만 사용
  '''
  if len(row["Predicted_a"]) > 20:
    dfa.at[index,'Predicted_a'] = row["Predicted_a"][:10]
    long_cnt +=1
    continue
    '''
 # score가 0.05점이 넘지 않는다면 빈칸으로 대체

  #예측의 마지막이 조사라면 조사를 제거
  tag = Okt().pos(row["Predicted_b"])
  if tag[-1][1] == 'Josa':
    target = tag[-1][0]
    clean = row["Predicted_b"].rstrip(target)
    dfb.at[index,'Predicted_b'] = clean
    remove_josa +=1

print("길어서 제거된 답 ",long_cnt)
print("score가 낮아서 제거된 답", low_score)
print("제거된 조사 수", remove_josa)

길어서 제거된 답  0
score가 낮아서 제거된 답 0
제거된 조사 수 30


In [9]:
dfb

,Predicted_b,Start,End,Score,Answer,Id
0,뉴 740Li 25주년 에디션,666.0,682.0,0.103343,"['뉴 740Li 25주년 에디션', '뉴 740Li 25주년']",d14cb73158624cf094c546d856fd3c80
1,뒤셀도르프,559.0,565.0,0.572797,['독일 뒤셀도르프'],906631384e91493ebe1c7f34aea6f241
2,꾸준히,1071.0,1074.0,0.981770,['페이스북'],35e61dcb479643448a2cb7d326ae50a6
3,꾸준히,1071.0,1074.0,0.983212,['페이스북'],075e761b370040cb9041eecd39afc27c
4,마드리드,408.0,413.0,0.980107,['마드리드'],e67ed38f3dd944be94d5b4c53731f334
...,...,...,...,...,...,...
4004,2만달러로,546.0,551.0,0.118583,['200만엔'],05fcb8054dc44dab8683579c2cf5e465
4005,중동 건설 현장,470.0,480.0,0.677122,"['중동 건설 현장', '중동']",cc7f826b66724ce9b39e3a974ca15661
4006,아시아~유럽,383.0,389.0,0.535192,"['아시아~유럽 노선', '아시아~유럽']",3282034aa41e4fab980851ffd4a868dd
4007,12일까지,988.0,993.0,0.612616,['사흘'],0a73550b36df4baf82ac2f98619d22e7


In [39]:
df1 = pd.merge(dfa, dfb, how='inner', on ='Id')
df1 = df1.rename(columns={'Answer_x':'Answer'})
df = df1
df

,Id,Predicted_a,Start_x,End_x,Score_x,Predicted_b,Start_y,End_y,Score_y,Answer
0,906631384e91493ebe1c7f34aea6f241,독일 뒤셀도르프,556.0,565.0,0.725427,뒤셀도르프,559.0,565.0,0.572797,['독일 뒤셀도르프']
1,35e61dcb479643448a2cb7d326ae50a6,페이스북,543.0,548.0,0.556682,꾸준히,1071.0,1074.0,0.981770,['페이스북']
2,075e761b370040cb9041eecd39afc27c,페이스북,543.0,548.0,0.500647,꾸준히,1071.0,1074.0,0.983212,['페이스북']
3,e67ed38f3dd944be94d5b4c53731f334,마드리드,408.0,413.0,0.998218,마드리드,408.0,413.0,0.980107,['마드리드']
4,80cacfdfe76442b0b39568e8b193d066,국제 원자재값 하락,824.0,835.0,0.588479,최소,1167.0,1169.0,0.936785,['국제 원자재값 하락']
...,...,...,...,...,...,...,...,...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,2만달러로,546.0,551.0,0.242327,2만달러로,546.0,551.0,0.118583,['200만엔']
4004,cc7f826b66724ce9b39e3a974ca15661,중동 건설 현장,470.0,480.0,0.512491,중동 건설 현장,470.0,480.0,0.677122,"['중동 건설 현장', '중동']"
4005,3282034aa41e4fab980851ffd4a868dd,아시아~유럽 노선,383.0,393.0,0.589738,아시아~유럽,383.0,389.0,0.535192,"['아시아~유럽 노선', '아시아~유럽']"
4006,0a73550b36df4baf82ac2f98619d22e7,사흘간,313.0,316.0,0.602099,12일까지,988.0,993.0,0.612616,['사흘']


In [40]:
answer = df.drop(['Predicted_a', 'Predicted_b','Start_x','Start_y','Predicted_b', 'End_x','End_y','Score_y', 'Score_x'], axis=1)
answer['predict'] = None
answer

,Id,Answer,predict
0,906631384e91493ebe1c7f34aea6f241,['독일 뒤셀도르프'],None
1,35e61dcb479643448a2cb7d326ae50a6,['페이스북'],None
2,075e761b370040cb9041eecd39afc27c,['페이스북'],None
3,e67ed38f3dd944be94d5b4c53731f334,['마드리드'],None
4,80cacfdfe76442b0b39568e8b193d066,['국제 원자재값 하락'],None
...,...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,['200만엔'],None
4004,cc7f826b66724ce9b39e3a974ca15661,"['중동 건설 현장', '중동']",None
4005,3282034aa41e4fab980851ffd4a868dd,"['아시아~유럽 노선', '아시아~유럽']",None
4006,0a73550b36df4baf82ac2f98619d22e7,['사흘'],None


In [38]:
# a가 성능이 가장 좋음(kobigbird)

for i in range(4008):
  num = 0
  # a, b, c의 값이 모두 같을 때
  if df.loc[i, 'Predicted_a'] == df.loc[i,'Predicted_b']:
    answer.loc[i, 'predict'] = df.loc[i, 'Predicted_a']
  
  # a, b 값이 다를 때
  elif df.loc[i, 'Predicted_a'] in df.loc[i,'Predicted_b']:
    answer.loc[i, 'predict'] = df.loc[i, 'Predicted_a']
  elif df.loc[i, 'Predicted_b'] in df.loc[i,'Predicted_a']:
    if df.loc[i, 'Score_x'] >= df.loc[i,'Score_y']:
      answer.loc[i, 'predict'] = df.loc[i, 'Predicted_a']
    else:
      answer.loc[i, 'predict'] = df.loc[i, 'Predicted_b']
  elif df.loc[i, 'Predicted_a'] != df.loc[i,'Predicted_b']:
    answer.loc[i, 'predict'] = df.loc[i, 'Predicted_a']

In [36]:
answer

,Id,Answer,predict
0,906631384e91493ebe1c7f34aea6f241,['독일 뒤셀도르프'],독일 뒤셀도르프
1,35e61dcb479643448a2cb7d326ae50a6,['페이스북'],페이스북
2,075e761b370040cb9041eecd39afc27c,['페이스북'],페이스북
3,e67ed38f3dd944be94d5b4c53731f334,['마드리드'],마드리드
4,80cacfdfe76442b0b39568e8b193d066,['국제 원자재값 하락'],국제 원자재값 하락
...,...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,['200만엔'],2만달러로
4004,cc7f826b66724ce9b39e3a974ca15661,"['중동 건설 현장', '중동']",중동 건설 현장
4005,3282034aa41e4fab980851ffd4a868dd,"['아시아~유럽 노선', '아시아~유럽']",아시아~유럽 노선
4006,0a73550b36df4baf82ac2f98619d22e7,['사흘'],사흘간


In [37]:
answer.to_csv('/content/drive/MyDrive/Colab Notebooks/ensemble_newww.csv')